In [1]:
import os
import pandas as pd
import numpy as np

import requests
import json
from pandas.io.json import json_normalize

In [2]:
path = r"/Users/jib/Documents/Projects/Hackathon"
os.chdir(path)
os.getcwd()

'/Users/jib/Documents/Projects/Hackathon'

# Library

In [220]:
appl = potential_ind('0203')
appl

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,ctyName_x,CCISO2_x,CCISO3,CTRYG_x,hs_code,ctyName_y,CCISO2_y,CTRYG_y,CAGR,wld_share,...,lpi_year,lpi_score,distance_from_th,fta_with_th,cte_2018,efi_score_2019,st_political risk,mlt_political risk,commercial risk,transfer restriction risk
0,Afghanistan,AF,AFG,AF,0203,Afghanistan,AF,AF,NaN,NaN,...,2018.0,1.948565,3884.74088,0,453.0,51.5,7,7,C,7


In [217]:
def potential_ind(hs_code):
    p_index = pd.read_csv('countrydict.csv')
    p_index['hs_code'] = str(hs_code)
    
    df_pd  = product_ind(hs_code)
    p_index = p_index.merge(df_pd, left_on=['CCISO3'], right_on=['CCISO3'], how="left")
    
    df_cnty = country_ind()
    p_index = p_index.merge(df_cnty, left_on=['CCISO3'], right_on=['CCISO3'], how="left")
    
    return p_index
    
def product_ind(hs_code):
    url = "http://comtrade.un.org/api/get?r=" + "All" + "&freq=" + "A" + "&ps=" + "2014,2015,2016,2017,2018" +"&px="+ "HS" + "&p=" + "0,764" + "&rg=" + "1" + "&cc=" + hs_code + "&fmt=" + "json"  + "&max=" + "10000" + "&type=" + "C" 
    url = requests.get(url)
    data = json.loads(url.text)
    
    df_im = json_normalize(data['dataset'])
    
    cn_list = pd.read_csv('countrydict.csv')
    
    cn_list = cn_list.merge(importcagr(df_im), left_on=['CCISO3'], right_on=['rt3ISO'], how="left").drop(columns=['rt3ISO'])
    cn_list = cn_list.merge(imwldshare(df_im), left_on=['CCISO3'], right_on=['rt3ISO'], how="left").drop(columns=['rt3ISO'])
    cn_list = cn_list.merge(imfromthshare(df_im), left_on=['CCISO3'], right_on=['rt3ISO'], how="left").drop(columns=['rt3ISO'])
    cn_list = cn_list.merge(tariffrate(hs_code), left_on=['CTRYG'], right_on=['cnty'], how="left").drop(columns=['cnty'])
    
    return cn_list[['CCISO3','CAGR', 'wld_share','wld_im_year','tha_share','im_from_th_year','AVDuty average']]
    
# 1. import from world growth (CAGR)
def importcagr(data):    
    df_im_w = data[data['pt3ISO'] == 'WLD']
    df_im_w = df_im_w[['rt3ISO','period','TradeValue']]
    
    pv = df_im_w.pivot_table(index=['rt3ISO'],columns=['period'],values=['TradeValue'],aggfunc='sum').fillna(0)
    pv = pv['TradeValue'].reset_index()
    pv['CAGR'] = (pv.iloc[:, [3,4,5]].sum(axis=1)).div(pv.iloc[:, [2,3,4]].sum(axis=1)).pow(1/(1)).sub(1)
    
    return pv[['rt3ISO','CAGR']]

# 2. import from world share (cnty_im/ world_im)
def imwldshare(data):
    df_im_w = data[data['pt3ISO'] == 'WLD']
    df_im_w = df_im_w[['rt3ISO','period','TradeValue']]

    pv = df_im_w.pivot_table(index=['period'],values=['TradeValue'],aggfunc='sum').fillna(0)
    pv = pv['TradeValue'].reset_index().rename(columns={"TradeValue":"wtradevalue"})

    yrmax = df_im_w[np.isnan(df_im_w['period']) == False].groupby('rt3ISO')['period'].max().reset_index().rename(columns = {'period':'yr_max'})
    df_im_w = df_im_w.merge(yrmax,on='rt3ISO',how = 'left')
    df_im_w = df_im_w[df_im_w['yr_max'] == df_im_w['period']].reset_index()
    df_im_w = df_im_w.merge(pv,how='left')

    # World share
    df_im_w['wld_share'] = df_im_w['TradeValue'].div(df_im_w['wtradevalue'])
    df_im_w = df_im_w.rename(columns={'period':'wld_im_year'})
    
    return df_im_w[['rt3ISO','wld_share','wld_im_year']]
    
# 3. import from Thai share (im_from_th / im_from_world)
def imfromthshare(data):
    df_im_w = data[data['pt3ISO'] == 'WLD']
    df_im_th = data[data['pt3ISO'] == 'THA']

    df_im_w = df_im_w[['rt3ISO','period','TradeValue']].rename(columns={'TradeValue':'wtradevalue'})
    df_im_th = df_im_th[['rt3ISO','period','TradeValue']].rename(columns={'TradeValue':'thtradevalue'})

    yrmax = df_im_w[np.isnan(df_im_w['period']) == False].groupby('rt3ISO')['period'].max().reset_index().rename(columns = {'period':'yr_max'})
    df_im_all = df_im_w.merge(yrmax,on='rt3ISO',how = 'left')
    df_im_all = df_im_all[df_im_all['yr_max'] == df_im_all['period']].reset_index()
    df_im_all = df_im_all.merge(df_im_th, on=['rt3ISO','period'], how='left')

    df_im_all['tha_share'] = df_im_all['thtradevalue'].div(df_im_all['wtradevalue'])
    df_im_all = df_im_all.rename(columns={'period':'im_from_th_year'})
    
    return df_im_all[['rt3ISO', 'tha_share', 'im_from_th_year']]

# 4. avg tariff rate
def tariffrate(hs_code):
    df_tr = pd.read_csv('TC_MFN.csv')
    if hs_code[0] == '0':
        hs_n = hs_code[1:]
    else:
        hs_n = hs_code
    df_tr = df_tr[(df_tr['Product'] == int(hs_n))][['cnty','AVDuty average']].reset_index()
    return df_tr[['cnty', 'AVDuty average']]

def country_ind():
    cn_ind = pd.read_csv('country_ind.csv')
    cn_ind = cn_ind.drop(['ctyName','CCISO2'], axis=1)
    return cn_ind

def normalized(series):
    mean = series.mean()
    std = series.std()
    zscore = (series - mean)/std
    return zscore

def tosubindex(series, weight):
    index = series.mul(weight)
    return index

# Backup

In [3]:
#### weight on

#market potential (50%)
w_im_growth = 0.1
w_im_share = 0.1
w_GDP = 0.1

#trade convenient (30%)
w_avg_tariff = 0.1 

w_lpi = 0.1
w_costtoex = 0.1
w_distances = 0.1
w_fta = 0.1

#country risk (20%)
w_political = 0.1
w_commercial = 0.1
w_corruption = 0.1